Охуительная идея для структуры голосования:

по строкам - два метода

по колонкам - разные ядра

в ячейках - количество выигранных раундов (1 раунд - 1 граф)

Какие графики мне нужны?

1) Набор графиков, на каждом по простой и лог мере и на последнем будет 3 меры, оба метода (для 3/4 семейств сгенерированных графов)

2) Как в 1), но для реальных датасетов

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.spectral_clustering import SpectralClustering
from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels

n_clusters = 4
generator = GraphGenerator(100, n_clusters, 0.2, 0.05)

for kernel_class in get_all_kernels():
    ward_results = {}
    spectral_results = {}
    all_ward_scores = []
    all_spectral_scores = []
    for i in range(0, 10):
        graph = generator.generate_graph()
        nodes = graph.nodes
        edges = graph.edges
        ward_scores = []
        spectral_scores = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            ward_prediction = Ward(n_clusters).fit_predict(K)
            ward_score = adjusted_rand_score(nodes, ward_prediction)
            ward_scores.append(ward_score)
            spectral_prediction = SpectralClustering(n_clusters).fit_predict(K)
            spectral_score = adjusted_rand_score(nodes, spectral_prediction)
            spectral_scores.append(spectral_score)
        all_ward_scores.append(ward_scores)
        all_spectral_scores.append(spectral_scores)
    all_ward_scores = [list(i) for i in zip(*all_ward_scores)]
    all_spectral_scores = [list(i) for i in zip(*all_spectral_scores)]
    resulting_ward_score = [np.mean(x) for x in all_ward_scores]
    resulting_spectral_score = [np.mean(x) for x in all_spectral_scores]
    ward_results[kernel_class.name] = resulting_ward_score
    spectral_results[kernel_class.name] = resulting_spectral_score
    
    with open('src/results/my/my_graphs/ward' + kernel_class.name + '.txt', 'w') as file:
        file.write(json.dumps(ward_results))
    with open('src/results/my/my_graphs/k-means' + kernel_class.name + '.txt', 'w') as file:
        file.write(json.dumps(spectral_results))
        
    plt.figure(figsize=(10,10))
    print('Ward ', kernel_class.name, 'best result: ', np.max(resulting_ward_score))
    print('K-means ', kernel_class.name, 'best result: ', np.max(resulting_spectral_score))
    for kernel_name, resulting_score in ward_results.items():
        print(kernel_name, 'best result: ', np.max(resulting_score))
        plt.plot(kernel_class.default_params, resulting_score, label='Ward')
    for kernel_name, resulting_score in spectral_results.items():
        print(kernel_name, 'best result: ', np.max(resulting_score))
        plt.plot(kernel_class.default_params, resulting_score, label='K-means')
    # plt.plot(kernel_class.default_params, resulting_spectral_score, label='K-means')
    plt.title(kernel_class.name + ', n_clusters=' + str(n_clusters))
    plt.legend()
    plt.show()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.spectral_clustering import SpectralClustering
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels


generator = GraphGenerator(200, 4, 0.2, 0.05)
n_clusters = 4

for kernel_class in get_all_kernels():
    results = {}
    all_scores = []
    for i in range(0, 50):
        graph = generator.generate_graph()
        nodes = graph.nodes
        edges = graph.edges
        scores = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            prediction = SpectralClustering(n_clusters).fit_predict(K)
            score = adjusted_rand_score(nodes, prediction)
            scores.append(score)
        all_scores.append(scores)
    all_scores = [list(i) for i in zip(*all_scores)]
    resulting_score = [np.mean(x) for x in all_scores]
    results[kernel_class.name] = resulting_score
    
    with open('src/results/my/' + kernel_class.name + '.txt', 'w') as file:
        file.write(json.dumps(results))
        
    plt.figure(figsize=(10,10))
    for kernel_name, resulting_score in results.items():
        print(kernel_name, 'best result: ', np.max(resulting_score))
        plt.plot(kernel_class.default_params, resulting_score, label=kernel_name)
    plt.title('K-means ' + kernel_class.name + ', n_clusters=' + str(n_clusters))
    plt.legend()
    plt.show()

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels
from src.dataset.dataset import football, polbooks, polblogs, zachary, news

all_data = news

for data in all_data:
    graph, info = data
    results = {}
    for kernel_class in get_all_kernels():
        results[kernel_class.name] = {}
        nodes = graph.nodes
        edges = graph.edges
        scores_1 = []
        scores_2 = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            prediction = Ward(info['k']).fit_predict(K)
            score_1 = adjusted_rand_score(nodes, prediction)
            scores_1.append(score_1)
            score_2 = normalized_mutual_info_score(nodes, prediction)
            scores_2.append(score_2)
        results[kernel_class.name] = str(max(scores_1)) + ' ' + str(max(scores_2))
        print(info['name'] + ' ' + kernel_class.name + ' ' + str(max(scores_1)) + ' ' + str(max(scores_2)))
    print('data ' + info['name'] + ' finished!')
    with open('src/results/my/news/' + info['name'] + '.txt', 'w') as file:
        file.write(json.dumps(results))

news_2cl_1 PlainWalk 0.046538240459 0.131073947536


news_2cl_1 LogPlainWalk 0.765036841281 0.666211167588


news_2cl_1 Communicability 0.0398139851986 0.137056806879


news_2cl_1 LogCommunicability 0.0398139851986 0.137056806879


news_2cl_1 Forest 0.0 0.0157732781716


...

news_3cl_1 LogPlainWalk 0.698476902125 0.626289453208


news_3cl_1 Communicability 0.396134440162 0.527740021915


news_3cl_1 LogCommunicability 0.396134440162 0.527740021915


news_3cl_1 Forest 1.1167038899e-05 0.0223206287314


news_3cl_1 LogForest 1.1167038899e-05 0.0223206287314


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels
from src.dataset.dataset import football, polbooks, polblogs, zachary, news

all_data = [football, polbooks, zachary, polblogs]

for data in all_data:
    graph, info = data
    results = {}
    for kernel_class in get_all_kernels():
        results[kernel_class.name] = {}
        nodes = graph.nodes
        edges = graph.edges
        scores_1 = []
        scores_2 = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            prediction = Ward(info['k']).fit_predict(K)
            score_1 = adjusted_rand_score(nodes, prediction)
            scores_1.append(score_1)
            score_2 = normalized_mutual_info_score(nodes, prediction)
            scores_2.append(score_2)
        results[kernel_class.name] = str(max(scores_1)) + ' ' + str(max(scores_2))
        print(info['name'] + ' ' + kernel_class.name + ' ' + str(max(scores_1)) + ' ' + str(max(scores_2)))
    print('data ' + info['name'] + ' finished!')
    with open('src/results/my/others/' + info['name'] + '.txt', 'w') as file:
        file.write(json.dumps(results))

football PlainWalk 0.906334157557 0.930810884185


football LogPlainWalk 0.896650009791 0.92419578687


football Communicability 0.909377088152 0.935845790439


football LogCommunicability 0.909377088152 0.935845790439


football Forest 0.896650009791 0.92419578687


...

zachary Forest 0.153463105478 0.272495191494


zachary LogForest 0.153463105478 0.272495191494


zachary Heat 1.0 1.0


zachary LogHeat 1.0 1.0
data zachary finished!


KeyboardInterrupt: 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.kernel_kmeans import KernelKMeans
from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels
from src.dataset.dataset import football, polbooks, polblogs, zachary, news

all_data = [football, polbooks, zachary]

for data in all_data:
    graph, info = data
    print('data ' + info['name'] + ' for Ward started!')
    results = {}
    for kernel_class in get_all_kernels():
        results[kernel_class.name] = {}
        nodes = graph.nodes
        edges = graph.edges
        scores_1 = []
        scores_2 = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            prediction = Ward(info['k']).fit_predict(K)
            score_1 = adjusted_rand_score(nodes, prediction)
            scores_1.append(score_1)
            score_2 = normalized_mutual_info_score(nodes, prediction)
            scores_2.append(score_2)
        results[kernel_class.name] = 'ARI=' + str(max(scores_1)) + ' NMI=' \
                                     + str(max(scores_2))
        print(info['name'] + ' ' + kernel_class.name + ' ' + results[kernel_class.name])
    print('data ' + info['name'] + 'for Ward' + ' finished!')
    with open('src/results/my/datasets/' + 'Ward_' + info['name'] + '.txt', 'w') as file:
        file.write(json.dumps(results))

data football for Ward started!


football PlainWalk ARI=0.906334157557 NMI=0.930810884185


KeyboardInterrupt: 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.kernel_kmeans import KernelKMeans
from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels
from src.dataset.dataset import football, polbooks, polblogs, zachary, news

# all_data = [football, polbooks, zachary]
all_data = [polbooks]

for data in all_data:
    graph, info = data
    print('data ' + info['name'] + ' started!')
    print('n_clusters = ' + str(info['k']))
    results = {}
    for kernel_class in get_all_kernels():
        results[kernel_class.name] = {}
        nodes = graph.nodes
        edges = graph.edges
        scores_1 = []
        scores_2 = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            prediction = KernelKMeans(n_clusters=info['k']).fit_predict(K)
            score_1 = adjusted_rand_score(nodes, prediction)
            scores_1.append(score_1)
            score_2 = normalized_mutual_info_score(nodes, prediction)
            scores_2.append(score_2)
        results[kernel_class.name] = 'ARI=' + str(max(scores_1)) + ' NMI=' + str(max(scores_2))
        print(info['name'] + ' ' + kernel_class.name + ' ' 
              + results[kernel_class.name])
    print('data ' + info['name'] + ' finished!')
    # with open('src/results/my/datasets/' + 'K_means_' + info['name'] + '.txt', 'w') as file:
    #     file.write(json.dumps(results))

data polbooks started!
n_clusters = 3


polbooks PlainWalk ARI=0.357089639519 NMI=0.419866966672


polbooks LogPlainWalk ARI=0.673934585831 NMI=0.586863138501


polbooks Communicability ARI=0.443585210413 NMI=0.466520079343


polbooks LogCommunicability ARI=0.360581185653 NMI=0.414122778039


...

polbooks LogForest ARI=0.682989290703 NMI=0.605116578434


polbooks Heat ARI=0.707819422326 NMI=0.640765763434


polbooks LogHeat ARI=0.726959025023 NMI=0.645147951749


polbooks SigmoidCommuteTime ARI=0.733839072062 NMI=0.678825087596


polbooks SigmoidCorrectedCommuteTime ARI=0.0 NMI=-1.11022302463e-06
data polbooks finished!


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.kernel_kmeans import KernelKMeans
from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels
from src.dataset.dataset import football, polbooks, polblogs, zachary, news

all_data = [football, polbooks, zachary]
all_data += news

for data in all_data:
    graph, info = data
    print('data ' + info['name'] + 'for K-means' + ' started!')
    print('n_clusters = ' + str(info['k']))
    kmeans_results = {}
    ward_results = {}
    for kernel_class in get_all_kernels():
        kmeans_results[kernel_class.name] = {}
        ward_results[kernel_class.name] = {}
        nodes = graph.nodes
        edges = graph.edges
        kmeans_ari_s = []
        kmeans_nmi_s = []
        ward_ari_s = []
        ward_nmi_s = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            kmeans_prediction = KernelKMeans(n_clusters=info['k']).fit_predict(K)
            kmeans_ari = adjusted_rand_score(nodes, kmeans_prediction)
            kmeans_ari_s.append(kmeans_ari)
            kmeans_nmi = normalized_mutual_info_score(nodes, kmeans_prediction)
            kmeans_nmi_s.append(kmeans_nmi)

            ward_prediction = Ward(n_clusters=info['k']).fit_predict(K)
            ward_ari = adjusted_rand_score(nodes, ward_prediction)
            ward_ari_s.append(ward_ari)
            ward_nmi = normalized_mutual_info_score(nodes, ward_prediction)
            ward_nmi_s.append(ward_nmi)

        kmeans_results[kernel_class.name] = 'ARI=' + str(max(kmeans_ari_s)) \
                                            + ' NMI=' + str(max(kmeans_nmi_s))
        ward_results[kernel_class.name] = 'ARI=' + str(max(ward_ari_s)) \
                                            + ' NMI=' + str(max(ward_nmi_s))
        print(info['name'] + ' K-means ' + kernel_class.name + ' ' 
              + kmeans_results[kernel_class.name])
        print(info['name'] + ' Ward ' + kernel_class.name + ' ' 
              + ward_results[kernel_class.name])
    print('data ' + info['name'] + 'for K-means' + ' finished!')
    with open('src/results/my/datasets/' + 'K_means_' + info['name'] + '.txt', 'w') as file:
        file.write(json.dumps(kmeans_results))
    with open('src/results/my/datasets/' + 'Ward_' + info['name'] + '.txt', 'w') as file:
        file.write(json.dumps(ward_results))

data footballfor K-means started!
n_clusters = 12


football K-means PlainWalk ARI=0.0227841318655 NMI=0.287787297427
football Ward PlainWalk ARI=0.906334157557 NMI=0.930810884185


football K-means LogPlainWalk ARI=0.840399203505 NMI=0.907550804448
football Ward LogPlainWalk ARI=0.896650009791 NMI=0.92419578687


football K-means Communicability ARI=0.876554844307 NMI=0.935285635642
football Ward Communicability ARI=0.909377088152 NMI=0.935845790439


football K-means LogCommunicability ARI=0.876622989595 NMI=0.91695911395
football Ward LogCommunicability ARI=0.909377088152 NMI=0.935845790439


...

news_3cl_1 K-means PlainWalk ARI=0.164771820542 NMI=0.219073128399
news_3cl_1 Ward PlainWalk ARI=0.294076407196 NMI=0.435537100686


news_3cl_1 K-means LogPlainWalk ARI=0.823569797941 NMI=0.754428265803
news_3cl_1 Ward LogPlainWalk ARI=0.698476902125 NMI=0.626289453208


news_3cl_1 K-means Communicability ARI=0.351448434994 NMI=0.453860686525
news_3cl_1 Ward Communicability ARI=0.396134440162 NMI=0.527740021915


news_3cl_1 K-means LogCommunicability ARI=0.359739912474 NMI=0.41661445159
news_3cl_1 Ward LogCommunicability ARI=0.396134440162 NMI=0.527740021915


KeyboardInterrupt: 